<a href="https://colab.research.google.com/github/RohanRaghav/Deeplearning/blob/main/22Bai71120exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
import numpy as np

# 1. Data Preparation

# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Flatten the images to 1D vectors of size 784 (28*28)
x_train = x_train.reshape((x_train.shape[0], 28 * 28))
x_test = x_test.reshape((x_test.shape[0], 28 * 28))

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# 2. Layerwise Pre-training

def build_autoencoder(input_dim, encoding_dim):
    input_img = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_img)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder, encoder

# Layerwise training parameters
layer_dims = [512, 256, 128]
input_dim = x_train.shape[1]
autoencoders = []
encoders = []
encoded_input = x_train

# Train each layer's autoencoder
for encoding_dim in layer_dims:
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)
    autoencoder.fit(encoded_input, encoded_input,
                    epochs=10,
                    batch_size=256,
                    shuffle=True,
                    verbose=1)
    encoded_input = encoder.predict(encoded_input)
    autoencoders.append(autoencoder)
    encoders.append(encoder)
    input_dim = encoding_dim
model = Sequential()

# Add pre-trained layers
input_dim = x_train.shape[1]
for encoder in encoders:
    # Get the output dimension of the encoder layer
    encoding_dim = encoder.output_shape[1]
    # Add the Dense layer with pre-trained weights
    model.add(Dense(encoding_dim, activation='relu', input_shape=(input_dim,)))
    model.layers[-1].set_weights(encoder.get_layer(index=1).get_weights())
    input_dim = encoding_dim

# Add the output layer
model.add(Dense(10, activation='softmax'))


# 4. Fine-tuning

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy:.4f}')

# Predict on the test set
predictions = model.predict(x_test)
print(f'Predicted label: {np.argmax(predictions[0])}, True label: {np.argmax(y_test[0])}')
print('Done by 22Bai71120')

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - loss: 0.2449
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - loss: 0.0877
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 0.0743
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - loss: 0.0701
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - loss: 0.0682
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 0.0671
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 0.0664
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 0.0659
Epoch 9/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 0.0654
Epoch 10/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 0.0651
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: -1978.3170
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: -36103.4336
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - loss: -120791.4375
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.1031 - loss: 212582.5312 - val_accuracy: 0.1086 - val_loss: 440.6948
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.1927 - loss: 218.8811 - val_accuracy: 0.4996 - val_loss: 73.9540
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.3681 - loss: 70.8127 - val_accuracy: 0.5523 - val_loss: 15.3452
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6415 - loss: 12.9049 - val_accuracy: 0.7090 - val_loss: 8.3215
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.6976 - loss: 9.8830 - val_accuracy: 0.6900 - val_loss: 12.8371
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7510 - loss: 6.1063 - val_accuracy: 0.8120 - val_loss: 2.6381
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7809 - loss: 4.6324 - val_accuracy: 0.7329 - val_loss: 6.8443
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.8045 - loss: